# Document Augmentation RAG with Question Generation

## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json
import re
from tqdm import tqdm

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로.

    Returns:
        str: 추출된 전체 텍스트.
    """
    # PDF 파일을 엽니다.
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 텍스트를 저장할 문자열 초기화

    # 각 페이지를 순회하면서 텍스트를 추출합니다.
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 페이지 가져오기
        text = page.get_text("text")  # 해당 페이지에서 텍스트 추출
        all_text += text  # 추출된 텍스트를 누적

    return all_text  # 전체 텍스트 반환

## Chunking the Extracted Text

In [4]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 지정된 수의 문자가 겹치도록 분할합니다.

    Args:
        text (str): 분할할 원본 텍스트.
        n (int): 각 청크의 문자 수.
        overlap (int): 청크 간 중첩되는 문자 수.

    Returns:
        List[str]: 분할된 텍스트 청크 리스트.
    """
    chunks = []  # 청크를 저장할 빈 리스트 초기화
    
    # (n - overlap)만큼 이동하며 텍스트를 분할
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])  # 현재 위치부터 n자까지 슬라이싱하여 추가

    return chunks  # 청크 리스트 반환

## Generating Questions for Text Chunks

In [5]:
def generate_questions(text_chunk, num_questions=5, model="gpt-4o-mini"):
    """
    주어진 텍스트 청크로부터 관련 질문들을 생성합니다.

    Args:
        text_chunk (str): 질문을 생성할 대상 텍스트 청크.
        num_questions (int): 생성할 질문의 개수.
        model (str): 사용할 언어 모델.

    Returns:
        List[str]: 생성된 질문 리스트.
    """
    # AI의 역할을 정의하는 시스템 프롬프트
    system_prompt = (
        "당신은 텍스트로부터 관련 질문을 생성하는 전문가입니다. "
        "제공된 텍스트를 바탕으로 그 내용에만 근거한 간결한 질문들을 생성하세요. "
        "핵심 정보와 개념에 초점을 맞추세요."
    )
    
    # 사용자 프롬프트: 텍스트와 함께 질문 생성 요청
    user_prompt = f"""
    다음 텍스트를 기반으로, 해당 텍스트만으로 답할 수 있는 서로 다른 질문 {num_questions}개를 생성하세요:

    {text_chunk}
    
    응답은 번호가 매겨진 질문 리스트 형식으로만 작성하고, 그 외 부가 설명은 하지 마세요.
    """
    
    # 모델 호출을 통해 질문 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 응답에서 질문 문자열 추출
    questions_text = response.choices[0].message.content.strip()
    questions = []

    # 줄 단위로 질문을 추출하고 정리
    for line in questions_text.split('\n'):
        # 번호 제거 및 양쪽 공백 제거
        cleaned_line = re.sub(r'^\d+\.\s*', '', line.strip())
        if cleaned_line and cleaned_line.endswith('?'):
            questions.append(cleaned_line)

    return questions

## Creating Embeddings for Text

In [6]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    지정된 모델을 사용하여 입력 텍스트에 대한 임베딩을 생성합니다.

    Args:
        text (str): 임베딩을 생성할 입력 텍스트 또는 텍스트 리스트.
        model (str): 사용할 임베딩 모델 이름.

    Returns:
        dict: 생성된 임베딩 정보를 포함한 OpenAI API의 응답 객체.
    """
    # 입력 텍스트에 대해 임베딩 생성 요청
    response = client.embeddings.create(
        model=model,
        input=text
    )

    # 응답 객체 반환
    return response

## Building a Simple Vector Store

In [7]:
class SimpleVectorStore:
    """
    NumPy를 사용한 간단한 벡터 저장소 구현 클래스입니다.
    """
    def __init__(self):
        """
        벡터 저장소를 초기화합니다.
        """
        self.vectors = []    # 임베딩 벡터들을 저장
        self.texts = []      # 원본 텍스트들을 저장
        self.metadata = []   # 텍스트에 대한 메타데이터 저장
    
    def add_item(self, text, embedding, metadata=None):
        """
        벡터 저장소에 항목을 추가합니다.

        Args:
            text (str): 원본 텍스트.
            embedding (List[float]): 임베딩 벡터.
            metadata (dict, optional): 추가 메타데이터 (기본값: None).
        """
        self.vectors.append(np.array(embedding))             # 벡터 추가
        self.texts.append(text)                              # 텍스트 추가
        self.metadata.append(metadata or {})                 # 메타데이터 추가
    
    def similarity_search(self, query_embedding, k=5):
        """
        쿼리 임베딩과 가장 유사한 항목을 검색합니다.

        Args:
            query_embedding (List[float]): 쿼리 벡터.
            k (int): 반환할 결과 수 (기본값: 5).

        Returns:
            List[Dict]: 상위 k개의 유사 항목. 텍스트, 메타데이터, 유사도 포함.
        """
        if not self.vectors:
            return []
        
        query_vector = np.array(query_embedding)
        similarities = []

        # 각 벡터와의 코사인 유사도 계산
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (np.linalg.norm(query_vector) * np.linalg.norm(vector))
            similarities.append((i, similarity))
        
        # 유사도 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 상위 k개 결과 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })
        
        return results

## Processing Documents with Question Augmentation

In [8]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200, questions_per_chunk=5):
    """
    문서를 처리하고, 각 청크에 대해 질문을 생성하여 벡터 저장소에 추가합니다.

    Args:
        pdf_path (str): PDF 파일 경로.
        chunk_size (int): 각 청크의 문자 수.
        chunk_overlap (int): 청크 간 중첩 문자 수.
        questions_per_chunk (int): 청크당 생성할 질문 수.

    Returns:
        Tuple[List[str], SimpleVectorStore]: 생성된 텍스트 청크 리스트와 벡터 저장소 객체.
    """
    print("PDF에서 텍스트 추출 중...")
    extracted_text = extract_text_from_pdf(pdf_path)
    
    print("텍스트 청크 분할 중...")
    text_chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"총 {len(text_chunks)}개의 텍스트 청크가 생성되었습니다.")
    
    vector_store = SimpleVectorStore()
    
    print("각 청크에 대해 임베딩 및 질문 생성 중...")
    for i, chunk in enumerate(tqdm(text_chunks, desc="청크 처리 중")):
        # 청크 임베딩 생성
        chunk_embedding_response = create_embeddings(chunk)
        chunk_embedding = chunk_embedding_response.data[0].embedding
        
        # 청크를 벡터 저장소에 추가
        vector_store.add_item(
            text=chunk,
            embedding=chunk_embedding,
            metadata={"type": "chunk", "index": i}
        )
        
        # 해당 청크 기반 질문 생성
        questions = generate_questions(chunk, num_questions=questions_per_chunk)
        
        # 각 질문에 대한 임베딩 생성 후 저장소에 추가
        for j, question in enumerate(questions):
            question_embedding_response = create_embeddings(question)
            question_embedding = question_embedding_response.data[0].embedding
            
            vector_store.add_item(
                text=question,
                embedding=question_embedding,
                metadata={
                    "type": "question",
                    "chunk_index": i,
                    "original_chunk": chunk
                }
            )
    
    return text_chunks, vector_store

## Extracting and Processing the Document

In [9]:
# PDF 파일 경로 정의
pdf_path = "dataset/AI_Understanding.pdf"

# 문서 처리: 텍스트 추출, 청크 분할, 질문 생성, 벡터 저장소 구축
text_chunks, vector_store = process_document(
    pdf_path, 
    chunk_size=1000,       # 각 청크는 1000자
    chunk_overlap=200,     # 청크 간 200자 중첩
    questions_per_chunk=3  # 청크당 질문 3개 생성
)

# 벡터 저장소에 저장된 항목 개수 출력
print(f"벡터 저장소에 저장된 항목 수: {len(vector_store.texts)}개")

PDF에서 텍스트 추출 중...
텍스트 청크 분할 중...
총 21개의 텍스트 청크가 생성되었습니다.
각 청크에 대해 임베딩 및 질문 생성 중...


청크 처리 중: 100%|██████████| 21/21 [01:25<00:00,  4.06s/it]

벡터 저장소에 저장된 항목 수: 84개


## Performing Semantic Search

In [10]:
def semantic_search(query, vector_store, k=5):
    """
    쿼리와 벡터 저장소를 이용한 의미 기반 검색을 수행합니다.

    Args:
        query (str): 사용자 검색 쿼리.
        vector_store (SimpleVectorStore): 검색 대상 벡터 저장소.
        k (int): 반환할 결과 개수.

    Returns:
        List[Dict]: 관련성 높은 상위 k개의 항목 (텍스트, 메타데이터, 유사도 포함).
    """
    # 쿼리 임베딩 생성
    query_embedding_response = create_embeddings(query)
    query_embedding = query_embedding_response.data[0].embedding
    
    # 벡터 저장소에서 유사한 항목 검색
    results = vector_store.similarity_search(query_embedding, k=k)
    
    return results

## Running a Query on the Augmented Vector Store

In [11]:
# 검증 데이터셋을 JSON 파일에서 불러옵니다.
with open('dataset/validation.json') as f:
    data = json.load(f)

# 첫 번째 쿼리를 추출합니다.
query = data[0]['question']

# 의미 기반 검색을 통해 관련 있는 콘텐츠를 찾습니다.
search_results = semantic_search(query, vector_store, k=5)

print("Query:", query)
print("\nSearch Results:")

# 검색 결과를 타입에 따라 분류합니다.
chunk_results = []
question_results = []

for result in search_results:
    if result["metadata"]["type"] == "chunk":
        chunk_results.append(result)
    else:
        question_results.append(result)

# 먼저 문서 청크 결과 출력
print("\nRelevant Document Chunks:")
for i, result in enumerate(chunk_results):
    print(f"Context {i + 1} (유사도: {result['similarity']:.4f}):")
    print(result["text"][:300] + "...")
    print("-----------------------------")

# 이어서 관련 질문 결과 출력
print("\nMatched Questions:")
for i, result in enumerate(question_results):
    print(f"Question {i + 1} (유사도: {result['similarity']:.4f}):")
    print(result["text"])
    chunk_idx = result["metadata"].get("chunk_index", "N/A")
    print(f"연결된 청크 인덱스: {chunk_idx}")
    print("-----------------------------")

Query: '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?

Search Results:

Relevant Document Chunks:

Matched Questions:
Question 1 (유사도: 0.6719):
설명 가능한 AI(XAI)의 목표는 무엇인가요?
연결된 청크 인덱스: 5
-----------------------------
Question 2 (유사도: 0.6670):
AI의 투명성과 설명 가능성을 높이는 것이 왜 중요한가요?
연결된 청크 인덱스: 4
-----------------------------
Question 3 (유사도: 0.6499):
설명 가능한 AI(XAI)의 주요 목표는 무엇인가요?
연결된 청크 인덱스: 14
-----------------------------
Question 4 (유사도: 0.5694):
인공 지능(AI)의 정의는 무엇인가요?
연결된 청크 인덱스: 0
-----------------------------
Question 5 (유사도: 0.5665):
사용자에게 AI 시스템을 제어할 수 있는 권한을 부여하는 것이 왜 중요한가요?
연결된 청크 인덱스: 18
-----------------------------


## Generating Context for Response

In [12]:
def prepare_context(search_results):
    """
    응답 생성을 위한 통합 컨텍스트를 구성합니다.

    Args:
        search_results (List[Dict]): 의미 기반 검색 결과.

    Returns:
        str: 결합된 전체 컨텍스트 문자열.
    """
    # 이미 포함된 청크 인덱스를 추적하기 위한 집합
    chunk_indices = set()
    context_chunks = []
    
    # 우선적으로 직접적으로 매칭된 문서 청크를 추가
    for result in search_results:
        if result["metadata"]["type"] == "chunk":
            chunk_idx = result["metadata"]["index"]
            if chunk_idx not in chunk_indices:
                chunk_indices.add(chunk_idx)
                context_chunks.append(f"Chunk {chunk_idx}:\n{result['text']}")
    
    # 질문이 참조하는 원본 청크도 추가 (중복 제외)
    for result in search_results:
        if result["metadata"]["type"] == "question":
            chunk_idx = result["metadata"]["chunk_index"]
            if chunk_idx not in chunk_indices:
                chunk_indices.add(chunk_idx)
                original_chunk = result["metadata"]["original_chunk"]
                question_text = result["text"]
                context_chunks.append(
                    f"Chunk {chunk_idx} (참조 질문: '{question_text}'):\n{original_chunk}"
                )
    
    # 모든 청크를 하나의 문자열로 결합
    full_context = "\n\n".join(context_chunks)
    return full_context

## Generating a Response Based on Retrieved Chunks


In [13]:
def generate_response(query, context, model="gpt-4o-mini"):
    """
    쿼리와 컨텍스트를 기반으로 AI 응답을 생성합니다.

    Args:
        query (str): 사용자의 질문.
        context (str): 벡터 저장소에서 검색된 문맥 정보.
        model (str): 사용할 언어 모델 이름.

    Returns:
        str: 생성된 응답 텍스트.
    """
    # 시스템 프롬프트: 반드시 주어진 문맥에 기반해 응답하도록 설정
    system_prompt = (
        "당신은 주어진 컨텍스트에 기반하여 엄격하게 응답하는 AI 어시스턴트입니다. "
        "제공된 문맥에서 직접적으로 답변을 도출할 수 없는 경우, 다음과 같이 응답하세요: "
        "'I do not have enough information to answer that.'"
    )
    
    # 사용자 프롬프트: 질문과 함께 문맥을 제공
    user_prompt = f"""
        문맥:
        {context}

        질문: {query}

        위에 제공된 문맥에만 근거하여 질문에 답하세요. 간결하고 정확하게 응답해 주세요.
    """
    
    # AI 모델을 호출하여 응답 생성
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # 생성된 응답만 반환
    return response.choices[0].message.content

## Generating and Displaying the Response

In [14]:
# 검색 결과로부터 응답 생성을 위한 컨텍스트를 준비합니다.
context = prepare_context(search_results)

# 쿼리와 문맥을 기반으로 AI 응답을 생성합니다.
response_text = generate_response(query, context)

# 쿼리와 응답 출력
print("\n질문(Query):", query)
print("\n응답(Response):")
print(response_text)


질문(Query): '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?

응답(Response):
설명 가능한 AI(Explainable AI, XAI)는 AI 시스템을 더욱 투명하고 이해하기 쉽게 만드는 것을 목표로 합니다. XAI 기술은 AI 모델이 의사 결정을 내리는 방식에 대한 인사이트를 제공하여 신뢰와 책임감을 향상시키기 위해 개발되고 있습니다. 이는 AI 시스템에 대한 신뢰를 구축하는 데 필수적이며, 사용자가 AI의 공정성과 정확성을 평가할 수 있도록 돕습니다.


## Evaluating the AI Response

In [15]:
def evaluate_response(query, response, reference_answer, model="gpt-4o-mini"):
    """
    AI가 생성한 응답을 기준 정답과 비교하여 평가합니다.

    Args:
        query (str): 사용자 질문.
        response (str): AI가 생성한 응답.
        reference_answer (str): 기준이 되는 정답.
        model (str): 평가에 사용할 언어 모델.

    Returns:
        str: 평가 결과 및 점수에 대한 설명.
    """
    # 평가 시스템용 시스템 프롬프트 정의
    evaluate_system_prompt = """당신은 AI 응답을 평가하는 지능형 평가 시스템입니다.
    
    AI 어시스턴트의 응답을 기준 정답과 비교하여 다음 기준으로 평가하세요:
    1. 사실성(Factual correctness) - 정확한 정보를 담고 있는가?
    2. 완전성(Completeness) - 기준 정답의 핵심 내용을 충분히 포함하고 있는가?
    3. 관련성(Relevance) - 질문에 직접적으로 응답하고 있는가?

    아래 기준에 따라 0 ~ 1 사이의 점수를 부여하세요:
    - 1.0: 내용과 의미가 완벽하게 일치함
    - 0.8: 아주 좋음, 약간의 누락 또는 차이 있음
    - 0.6: 좋음, 주요 내용을 담고 있으나 일부 세부 정보 부족
    - 0.4: 부분적인 응답, 중요한 내용이 빠짐
    - 0.2: 관련된 정보가 거의 없음
    - 0.0: 틀리거나 전혀 관련 없는 응답

    점수와 함께 평가 사유를 제시하세요.
    """
    
    # 평가 요청용 프롬프트 구성
    evaluation_prompt = f"""
    사용자 질문:
    {query}

    AI 응답:
    {response}

    기준 정답:
    {reference_answer}

    위 기준에 따라 AI 응답을 평가해 주세요.
    """
    
    # 평가 모델 호출
    eval_response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": evaluate_system_prompt},
            {"role": "user", "content": evaluation_prompt}
        ]
    )
    
    # 평가 결과 반환
    return eval_response.choices[0].message.content

## Running the Evaluation

In [16]:
# 검증 데이터에서 기준 정답을 가져옵니다.
reference_answer = data[0]['ideal_answer']

# 생성된 응답을 기준 정답과 비교하여 평가합니다.
evaluation = evaluate_response(query, response_text, reference_answer)

# 평가 결과 출력
print("\nEvaluation:")
print(evaluation)


Evaluation:
점수: 0.8

평가 사유: AI 응답은 설명 가능한 AI(XAI)의 정의와 중요성을 잘 설명하고 있으며, 기준 정답의 핵심 내용을 대부분 포함하고 있습니다. 그러나 "AI 모델이 의사 결정을 내리는 방식에 대한 인사이트를 제공"이라는 표현은 다소 모호하게 느껴질 수 있으며, "공정성과 정확성을 평가할 수 있도록 돕는다"는 부분이 기준 정답에서 강조된 "공정성 보장"과는 약간의 차이가 있습니다. 따라서 약간의 누락이 있지만 전반적으로 매우 좋은 응답입니다.


## Extracting and Chunking Text from a PDF File

In [17]:
# PDF 파일 경로를 정의합니다.
pdf_path = "dataset/AI_Understanding.pdf"

# PDF 파일에서 텍스트를 추출합니다.
extracted_text = extract_text_from_pdf(pdf_path)

# 텍스트를 1000자 단위, 200자 중첩으로 청크 분할합니다.
text_chunks = chunk_text(extracted_text, 1000, 200)

# 생성된 텍스트 청크 수 출력
print("생성된 텍스트 청크 수:", len(text_chunks))

# 첫 번째 청크 출력
print("\n첫 번째 텍스트 청크:")
print(text_chunks[0])

생성된 텍스트 청크 수: 21

첫 번째 텍스트 청크:
인공 지능 이해 
 
1장: 인공 지능 소개 
인공 지능(AI)은 디지털 컴퓨터 또는 컴퓨터로 제어되는 로봇이 지적인 존재와 일반적으로 
관련된 작업을 수행할 수 있는 능력을 말합니다. 이 용어는 추론, 의미 발견, 일반화, 과거 
경험으로부터의 학습 능력 등 인간의 특징적인 지적 프로세스가 부여된 시스템을 
개발하는 프로젝트에 자주 적용됩니다. 지난 수십 년 동안 컴퓨팅 성능과 데이터 가용성의 
발전으로 AI의 개발과 배포가 크게 가속화되었습니다. 
역사적 맥락 
인공 지능에 대한 개념은 수세기 동안 존재해 왔으며 종종 신화와 소설에 묘사되기도 
했습니다. 하지만 공식적인 AI 연구 분야는 20세기 중반에 시작되었습니다. 1956년 
다트머스 워크숍은 AI의 발상지로 널리 알려져 있습니다. 초기 AI 연구는 문제 해결과 
상징적 방법에 중점을 두었습니다. 1980년대에는 전문가 시스템이 등장했고, 1990년대와 
2000년대에는 머신러닝과 신경망이 발전했습니다. 최근 딥러닝의 획기적인 발전은 이 
분야에 혁신을 가져왔습니다. 
현대 관측 
최신 AI 시스템은 일상 생활에서 점점 더 널리 보급되고 있습니다. Siri와 Alexa 같은 가상 
비서부터 스트리밍 서비스 및 소셜 미디어의 추천 알고리즘에 이르기까지 AI는 우리의 
생활, 업무, 상호 작용 방식에 영향을 미치고 있습니다. 자율 주행 자동차, 첨단 의료 진단, 
정교한 재무 모델링 도구의 개발은 AI가 광범위하고 성장하는 응용 분야를 보여줍니다. 
윤리적 영향, 편견, 일자리 대체에 대한 우려도 점점 더 커지고 있습니다. 
 
2장: 인공 지능의 핵심 개념 
머신 러닝 
머신러닝(ML)은 명시적으로 프로그래밍하지 않고도 시스템이 데이터로부터 학습할 수 
있도록 하는 데 중점을 둔 AI의 하위 집합입니다. ML 알고리즘은 더 많은 데이터에 
노출됨에 따라 패턴을 식별하고 예측하며 시간이 지남에 따라 성능을 개선합니다. 
지도 학습 
지도 학습에서는 

## Creating Embeddings for Text Chunks

In [18]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    지정된 모델을 사용하여 입력 텍스트에 대한 임베딩을 생성합니다.

    Args:
        text (str): 임베딩을 생성할 입력 텍스트 또는 텍스트 리스트.
        model (str): 사용할 임베딩 모델.

    Returns:
        dict: 생성된 임베딩 정보를 포함한 응답 객체.
    """
    # 지정된 모델로 임베딩 생성 요청
    response = client.embeddings.create(
        model=model,
        input=text
    )

    # 응답 객체 반환
    return response

# 텍스트 청크에 대해 임베딩을 생성합니다.
response = create_embeddings(text_chunks)

## Performing Semantic Search

In [19]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
        vec1 (np.ndarray): 첫 번째 벡터.
        vec2 (np.ndarray): 두 번째 벡터.

    Returns:
        float: 두 벡터 간의 코사인 유사도 값.
    """
    # 두 벡터의 내적을 계산하고, 벡터의 크기(norm) 곱으로 나누어 코사인 유사도를 구함
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [20]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    주어진 쿼리와 임베딩을 기반으로 텍스트 청크에서 의미 기반 검색을 수행합니다.

    Args:
        query (str): 의미 검색에 사용할 쿼리.
        text_chunks (List[str]): 검색 대상이 되는 텍스트 청크 리스트.
        embeddings (List[dict]): 각 텍스트 청크에 대한 임베딩 리스트.
        k (int): 반환할 관련 청크의 수 (기본값: 5).

    Returns:
        List[str]: 쿼리와 가장 관련성 높은 상위 k개의 텍스트 청크 리스트.
    """
    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []  # 유사도 점수를 저장할 리스트 초기화

    # 각 텍스트 청크 임베딩과 쿼리 임베딩 간의 유사도 계산
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(
            np.array(query_embedding),
            np.array(chunk_embedding.embedding)
        )
        similarity_scores.append((i, similarity_score))  # (인덱스, 유사도) 저장

    # 유사도 기준 내림차순 정렬
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # 상위 k개의 인덱스를 추출
    top_indices = [index for index, _ in similarity_scores[:k]]

    # 해당 인덱스의 텍스트 청크들을 반환
    return [text_chunks[index] for index in top_indices]

## Running a Query on Extracted Chunks

In [21]:
# 검증 데이터를 JSON 파일에서 불러옵니다.
with open('dataset/validation.json') as f:
    data = json.load(f)

# 첫 번째 쿼리를 추출합니다.
query = data[0]['question']

# 의미 기반 검색을 수행하여 쿼리와 가장 관련성 높은 상위 2개의 텍스트 청크를 찾습니다.
top_chunks = semantic_search(query, text_chunks, response.data, k=2)

# 쿼리를 출력합니다.
print("질문(Query):", query)

# 상위 2개의 관련 텍스트 청크를 출력합니다.
for i, chunk in enumerate(top_chunks):
    print(f"\n컨텍스트 {i + 1}:\n{chunk}\n{'-' * 37}")

질문(Query): '설명 가능한 AI(Explainable AI)'란 무엇이며, 왜 중요한가?

컨텍스트 1:
고 사회에 유익한 AI 시스템의 개발과 
배포를 보장하기 위한 지침입니다. 주요 원칙에는 인권 존중, 개인정보 보호, 비차별, 
공익성이 포함됩니다. 
AI의 편향성 해결 
AI 시스템은 데이터에 존재하는 편견을 유전하고 증폭시켜 불공정하거나 차별적인 결과를 
초래할 수 있습니다. 편향성을 해결하려면 신중한 데이터 수집, 알고리즘 설계, 지속적인 
모니터링 및 평가가 필요합니다. 
투명성 및 설명 가능성 
투명성과 설명 가능성은 AI 시스템에 대한 신뢰를 구축하는 데 필수적입니다. 설명 가능한 
AI(XAI) 기술은 AI의 결정을 더 이해하기 쉽게 만들어 사용자가 공정성과 정확성을 
평가할 수 있도록 하는 것을 목표로 합니다. 
개인정보 및 데이터 보호 
AI 시스템은 대량의 데이터에 의존하는 경우가 많기 때문에 개인정보 보호와 데이터 
보호에 대한 우려가 제기됩니다. 책임감 있는 데이터 처리, 개인정보 보호 기술 구현, 
데이터 보호 규정 준수는 매우 중요합니다. 
책임과 의무 
AI 시스템에 대한 책임과 의무를 확립하는 것은 잠재적인 피해를 해결하고 윤리적 행동을 
보장하는 데 필수적입니다. 여기에는 AI 시스템의 개발자, 배포자, 사용자에 대한 역할과 
책임을 정의하는 것이 포함됩니다. 
 
20장: AI에 대한 신뢰 구축 
투명성 및 설명 가능성 
투명성과 설명 가능성은 AI에 대한 신뢰를 구축하는 데 있어 핵심입니다. AI 시스템을 
이해하기 쉽게 만들고 의사 결정 프로세스에 대한 인사이트를 제공하면 사용자가 AI의 
신뢰성과 공정성을 평가하는 데 도움이 됩니다. 
견고성 및 신뢰성 
AI 시스템의 견고성과 신뢰성을 확보하는 것은 신뢰를 구축하는 데 필수적입니다. 
여기에는 AI 모델 테스트 및 검증, 성능 모니터링, 잠재적인 취약점 해결이 포함됩니다. 
사용자 제어 및 대행사 
사용자에게 AI 시스템을 제어할 수 있는 권한을 부여하고 AI

## Generating a Response Based on Retrieved Chunks

In [22]:
# AI 어시스턴트를 위한 시스템 프롬프트 정의
system_prompt = (
    "당신은 주어진 컨텍스트에 기반하여 엄격하게 응답하는 AI 어시스턴트입니다. "
    "제공된 문맥에서 직접적으로 답변을 도출할 수 없는 경우, 다음과 같이 응답하세요: "
    "'I do not have enough information to answer that.'"
)

def generate_response(system_prompt, user_message, model="gpt-4o-mini"):
    """
    시스템 프롬프트와 사용자 메시지를 기반으로 AI 모델의 응답을 생성합니다.

    Args:
        system_prompt (str): AI의 응답 방식을 정의하는 시스템 프롬프트.
        user_message (str): 사용자 메시지 또는 질문.
        model (str): 사용할 언어 모델.

    Returns:
        dict: AI 모델의 응답 객체.
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 검색된 상위 청크들을 바탕으로 사용자 프롬프트 생성
user_prompt = "\n".join([
    f"Context {i + 1}:\n{chunk}\n=====================================\n"
    for i, chunk in enumerate(top_chunks)
])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# AI 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

## Evaluating the AI Response

In [23]:
# 평가 시스템을 위한 시스템 프롬프트 정의
evaluate_system_prompt = (
    "당신은 AI 어시스턴트의 응답을 평가하는 지능형 평가 시스템입니다. "
    "AI 응답이 기준 정답과 매우 유사하면 점수 1을 부여하세요. "
    "응답이 부정확하거나 부적절하면 점수 0을 부여하세요. "
    "부분적으로 일치하면 점수 0.5를 부여하세요."
)

# 사용자 쿼리, AI 응답, 기준 정답을 포함한 평가용 프롬프트 생성
evaluation_prompt = (
    f"User Query: {query}\n"
    f"AI Response:\n{ai_response.choices[0].message.content}\n"
    f"True Response: {data[0]['ideal_answer']}\n"
    f"{evaluate_system_prompt}"
)

# 평가 수행
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 평가 점수 출력
print(evaluation_response.choices[0].message.content)

점수 1
